<a href="https://colab.research.google.com/github/Chuck2Win/NaverMovieSentiment/blob/master/%5B%EB%84%A4%EC%9D%B4%EB%B2%84_%EC%98%81%ED%99%94_%EB%A6%AC%EB%B7%B0_%EA%B0%90%EC%84%B1_%EB%B6%84%EC%84%9D%5D%5B%EC%A0%84%EC%B2%98%EB%A6%AC%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import re
import urllib.request
import os
from sklearn.model_selection import train_test_split
import sentencepiece as spm

Mounted at /content/gdrive


# 네이버 감성 분석 데이터 전처리
1. 한글이 아닌 부분 정제  
2. train, val, test set으로 나누기  
3. sentencepiece로 tokenizer 구축하기 ( vocab size 32000 ) 

In [ ]:
# 네이버 감성 분석 데이터 불러오기
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt", filename="./ratings_train.txt")
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt", filename="./ratings_test.txt")

('./ratings_test.txt', <http.client.HTTPMessage at 0x7f9849ceb278>)

In [ ]:
train_data = pd.read_table('./ratings_train.txt')
test_data = pd.read_table('./ratings_test.txt')

In [ ]:
# train, validation dataset 만들기
train_data,val_data=train_test_split(train_data,test_size=30000)

In [ ]:
print(len(train_data))
print(len(val_data))
print(len(test_data))

120000
30000
50000


In [ ]:
# 한글이 아닌 부분은 과감히 제거
def cleanse(sentence):
    result=sentence.lower()
    result=re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힇 ]','',result)
    return result

# 결측치가 있긴 하지만, 실제 test에서도 혹은 추후 데이터에서 결측치가 있을수도 있다고 생각해서 일단 학습
train_data['document']=train_data['document'].astype(str)
val_data['document']=val_data['document'].astype(str)
test_data['document']=test_data['document'].astype(str)

# 정제 작업
train_data['document']=train_data['document'].apply(lambda i : cleanse(i))
val_data['document']=val_data['document'].apply(lambda i : cleanse(i))
test_data['document']=test_data['document'].apply(lambda i : cleanse(i))

# 그리고 csv로 저장
train_data[['document','label']].to_csv('./Train_data.csv',index=False)
val_data[['document','label']].to_csv('./Val_data.csv',index=False)
test_data[['document','label']].to_csv('./Test_data.csv',index=False)

In [ ]:
# sentence piece
# sentence piece 구축
with open('naver_review.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(train_data['document'].astype(str)))

In [ ]:
spm.SentencePieceTrainer.Train('--input=naver_review.txt --model_prefix=naver_32000 --vocab_size=32000 --model_type=bpe --max_sentence_length=9999 --pad_id=0 --pad_piece=[PAD] --unk_id=1 --unk_piece=[UNK] --bos_id=2 --bos_piece=[BOS] --eos_id=3 --eos_piece=[EOS]')